In [33]:
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [5]:
df = pd.read_csv('../models/data/combined_openaq_weather.csv')
air_quality_params = ['co', 'no', 'no2', 'nox', 'o3', 'pm10', 'pm25', 'so2']
weather_features = ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'coco']
time_columns = ['time']
df.drop(columns=['datetimeLocal'], inplace=True)

In [12]:
parameter_dataframes = {}
for param in air_quality_params:
    selected_columns = time_columns + [param] + weather_features
    param_df = df[selected_columns].copy()
    param_df = param_df.dropna(subset=[param])
    parameter_dataframes[param] = param_df\


dfs=parameter_dataframes

params = ['co', 'no', 'no2', 'nox', 'o3', 'pm10', 'pm25', 'so2']
weather_cols = ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'coco']

In [15]:
#Feature selection using XGBoost
def xgb_feature_select(dfs, params, weather_cols, thresh=0.01,
                       n_estimators=100, random_state=42, verbose=True):
    selected = {}
    for p in params:
        df = dfs[p]
        model = xgb.XGBRegressor(n_estimators=n_estimators, random_state=random_state, verbosity=0)
        model.fit(df[weather_cols], df[p])
        imps = model.feature_importances_
        keep = [f for f, w in zip(weather_cols, imps) if w > thresh] or [weather_cols[int(imps.argmax())]]
        selected[p] = df[['time', p] + keep]
        if verbose:
            print(f"{p.upper():>6} | orig={len(weather_cols):2d}  keep={len(keep):2d}  feats={keep}")
    return selected

# Usage
selected_dfs = xgb_feature_select(dfs, params, weather_cols, thresh=0.01)


    CO | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
    NO | orig=11  keep= 8  feats=['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
   NO2 | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
   NOX | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
    O3 | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
  PM10 | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
  PM25 | orig=11  keep= 9  feats=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'coco']
   SO2 | orig=11  keep= 2  feats=['temp', 'dwpt']


In [18]:
#Removing outliers using z score method
total_dropped = 0
for p in params:
    df = selected_dfs[p]
    x = df[p].astype(float)
    m, sd = x.mean(), x.std(ddof=0)

    if sd == 0 or not np.isfinite(sd):
        dropped = 0
        keep = np.ones(len(df), dtype=bool)
    else:
        z = (x - m) / sd
        keep = np.abs(z) <= 3
        dropped = int((~keep).sum())

    selected_dfs[p] = df[keep].reset_index(drop=True)
    total_dropped += dropped
    print(f"{p.upper()}: dropped {dropped} rows")

print(f"TOTAL dropped: {total_dropped}")

CO: dropped 113 rows
NO: dropped 63 rows
NO2: dropped 113 rows
NOX: dropped 87 rows
O3: dropped 12 rows
PM10: dropped 35 rows
PM25: dropped 39 rows
SO2: dropped 33 rows
TOTAL dropped: 495


In [23]:
#Removing features with high multicollinearity using VIF
def drop_high_vif(cleaned_dfs, params, thresh=10.0):
    for p in params:
        df = cleaned_dfs[p]
        feats = (df.drop(columns=[c for c in ['time', p] if c in df.columns])
                   .select_dtypes(include=[np.number]))
        feats = feats.loc[:, feats.std(ddof=0) > 0]
        dropped = []
        while feats.shape[1] > 1:
            X = feats.replace([np.inf, -np.inf], np.nan).fillna(feats.median(numeric_only=True))
            vifs = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
            bad = vifs.idxmax()
            if vifs[bad] <= thresh: break
            feats = feats.drop(columns=[bad]); dropped.append(f"{bad}({vifs[bad]:.1f})")
        cleaned_dfs[p] = df.drop(columns=[c.split('(')[0] for c in dropped], errors='ignore')
        print(f"{p.upper()}: dropped {len(dropped)} → {', '.join(dropped) if dropped else 'none'}")

# Usage
drop_high_vif(selected_dfs, params, thresh=10.0)


CO: dropped 2 → pres(156.6), wpgt(39.9)
NO: dropped 2 → pres(156.4), wpgt(40.0)
NO2: dropped 2 → pres(156.7), wpgt(39.9)
NOX: dropped 2 → pres(156.8), wpgt(39.9)
O3: dropped 2 → pres(157.6), wpgt(40.3)
PM10: dropped 2 → pres(162.5), wpgt(40.9)
PM25: dropped 2 → pres(157.3), wpgt(40.2)
SO2: dropped 0 → none


In [27]:
def skew_check_and_log_transform(final_dfs, params, thresh=1.0):
    transformed_dfs, rows = {}, []
    for p in params:
        df = final_dfs[p].copy()
        x = df[p].astype(float)
        sk0 = float(skew(x, nan_policy='omit'))
        do_log = abs(sk0) >= thresh

        if do_log:
            shift = max(0.0, 1.0 - x.min())  # ensure strictly >0 before log
            x_pos = x + shift
            df[f'{p}_log'] = np.log(x_pos)
            sk1 = float(skew(df[f'{p}_log'], nan_policy='omit'))
        else:
            shift, sk1 = 0.0, np.nan

        transformed_dfs[p] = df
        rows.append({
            "param": p.upper(),
            "skew": sk0,
            "log_applied": do_log,
            "shift_used": shift if do_log else 0.0,
            "skew_after": sk1 if do_log else None,
            "improve": (abs(sk0) - abs(sk1)) if do_log else None
        })

    summary = pd.DataFrame(rows).sort_values("skew", key=lambda s: s.abs(), ascending=False)
    print(summary.to_string(index=False))
    return transformed_dfs, summary

# Usage:
transformed_dfs, skew_summary = skew_check_and_log_transform(selected_dfs, params, thresh=1.0)


param      skew  log_applied  shift_used  skew_after  improve
 PM10  3.502768         True         0.0    0.210075 3.292693
   NO  3.241736         True         1.0    3.229749 0.011987
  NOX  2.100233         True         1.0    2.083399 0.016834
 PM25  2.080380         True         0.9    0.509982 1.570398
  NO2  1.667036         True         1.0    1.656979 0.010057
   O3 -0.503169        False         0.0         NaN      NaN
  SO2  0.347757        False         0.0         NaN      NaN
   CO -0.033095        False         0.0         NaN      NaN


In [31]:
def minmax_scale_all(transformed_dfs, params):
    scaled_dfs, scalers, summary = {}, {}, []
    for p in params:
        df = transformed_dfs[p]
        target = f"{p}_log" if f"{p}_log" in df.columns else p
        feats = [c for c in df.columns if c not in ('time', p) and not c.endswith('_shifted')]
        sc = MinMaxScaler().fit(df[feats]) if feats else None
        scaled = (pd.DataFrame(sc.transform(df[feats]), columns=[f"{c}_scaled" for c in feats], index=df.index)
                  if feats else pd.DataFrame(index=df.index))
        scaled_df = pd.concat([df[['time', target]], scaled], axis=1)
        scaled_dfs[p], scalers[p] = scaled_df, {"scaler": sc, "feature_names": feats}
        print(f"{p.upper()}: scaled {len(feats)} features → {scaled_df.shape}")
        summary.append((p.upper(), target, len(feats)))
    print("\nSummary:", *[f"{p}: {n} feats, target={t}" for p,t,n in summary], sep="\n")
    return scaled_dfs, scalers

# Usage:
scaled_dfs, scalers = minmax_scale_all(transformed_dfs, params)

#Apply sclaer at inference TIME

CO: scaled 7 features → (4776, 9)
NO: scaled 7 features → (4826, 9)
NO2: scaled 8 features → (4776, 10)
NOX: scaled 8 features → (4802, 10)
O3: scaled 7 features → (4877, 9)
PM10: scaled 8 features → (4854, 10)
PM25: scaled 8 features → (4850, 10)
SO2: scaled 2 features → (4856, 4)

Summary:
CO: 7 feats, target=co
NO: 7 feats, target=no_log
NO2: 8 feats, target=no2_log
NOX: 8 feats, target=nox_log
O3: 7 feats, target=o3
PM10: 8 feats, target=pm10_log
PM25: 8 feats, target=pm25_log
SO2: 2 feats, target=so2


In [34]:
def train_simple_xgb(dfs, params, weather_cols, test_frac=0.2, n_estimators=100, random_state=42):
    """
    dfs[param] must have columns: ['time', param] + weather_cols (hourly or at least sorted by time).
    Features = weather_cols + f'{param}_lag1'  (lag is t-1h). Target = param at time t.
    """
    results = {}
    for p in params:
        df = dfs[p].copy()
        if 'time' in df.columns:
            df = df.sort_values('time').set_index('time')
        df[f'{p}_lag1'] = df[p].shift(1)

        feats = weather_cols + [f'{p}_lag1']
        tmp = df[feats + [p]].dropna()   # drop rows where lag1 or weather is missing
        if len(tmp) < 100:
            results[p] = {"note": "not enough data"}; continue

        split = int(len(tmp) * (1 - test_frac))
        Xtr, Xte = tmp.iloc[:split][feats], tmp.iloc[split:][feats]
        ytr, yte = tmp.iloc[:split][p],     tmp.iloc[split:][p]

        model = xgb.XGBRegressor(
            n_estimators=n_estimators, learning_rate=0.05, max_depth=6,
            subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
            random_state=random_state, verbosity=0
        )
        model.fit(Xtr, ytr, eval_set=[(Xte, yte)], verbose=False)

        yhat = pd.Series(model.predict(Xte), index=yte.index)
        results[p] = {
            "model": model,
            "features": feats,
            "mae": float(mean_absolute_error(yte, yhat)),
            "r2":  float(r2_score(yte, yhat))
        }
        print(f"{p.upper()}: MAE={results[p]['mae']:.3f} | R2={results[p]['r2']:.3f}")
    return results

def predict_current(dfs, param, weather_cols, pack):
    """Predict param at the latest time t using weather@t + lag1."""
    df = dfs[param].copy()
    if 'time' in df.columns:
        df = df.sort_values('time').set_index('time')
    df[f'{param}_lag1'] = df[param].shift(1)
    feats = pack["features"]
    x = df.iloc[[-1]][feats].dropna(axis=1, how='all')  # use latest row
    return float(pack["model"].predict(x)[0])

# Usage:
results = train_simple_xgb(scaled_dfs, params, weather_cols)
yhat_pm25_now = predict_current(scaled_dfs, 'pm25', weather_cols, results['pm25'])


KeyError: "['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'coco'] not in index"

In [ ]:
# Simple XGBoost with 1-hour lag features
def create_lag_features_simple(scaled_dfs, params):
    """Create 1-hour lag features for weather and target parameter"""
    lag_dfs = {}
    
    for param in params:
        df = scaled_dfs[param].copy()
        
        # Convert time to datetime and sort
        df['time'] = pd.to_datetime(df['time'])
        df = df.sort_values('time').reset_index(drop=True)
        
        # Get target column (log-transformed if available)
        target_col = f'{param}_log' if f'{param}_log' in df.columns else param
        
        # Create 1-hour lag for target parameter
        df[f'{target_col}_lag1'] = df[target_col].shift(1)
        
        # Create 1-hour lag for all scaled weather features
        weather_features = [col for col in df.columns if col.endswith('_scaled')]
        for feature in weather_features:
            df[f'{feature}_lag1'] = df[feature].shift(1)
        
        # Add basic time features
        df['hour'] = df['time'].dt.hour
        df['day_of_week'] = df['time'].dt.dayofweek
        df['month'] = df['time'].dt.month
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        
        # Store the enhanced dataframe
        lag_dfs[param] = df
        
        print(f"{param.upper()}: Added lag features → {df.shape}")
    
    return lag_dfs

# Create lag features
lag_dfs = create_lag_features_simple(scaled_dfs, params)

In [ ]:
# Train Simple XGBoost Models with 1-hour lag
def train_simple_lag_models(lag_dfs, params):
    """Train XGBoost models using current weather + 1-hour lag features"""
    simple_models = {}
    
    for param in params:
        print(f"\n🔄 Training {param.upper()} model...")
        
        df = lag_dfs[param].copy()
        
        # Get target column
        target_col = f'{param}_log' if f'{param}_log' in df.columns else param
        
        # Select features: current weather + lag features + time features
        feature_cols = []
        
        # Current scaled weather features
        current_weather = [col for col in df.columns if col.endswith('_scaled') and '_lag1' not in col]
        feature_cols.extend(current_weather)
        
        # 1-hour lag weather features
        lag_weather = [col for col in df.columns if col.endswith('_scaled_lag1')]
        feature_cols.extend(lag_weather)
        
        # 1-hour lag target parameter
        target_lag = f'{target_col}_lag1'
        if target_lag in df.columns:
            feature_cols.append(target_lag)
        
        # Time features
        time_features = ['hour', 'day_of_week', 'month', 'is_weekend']
        feature_cols.extend(time_features)
        
        # Remove rows with NaN (due to lag features)
        df_clean = df.dropna(subset=feature_cols + [target_col])
        
        if len(df_clean) < 100:
            print(f"   ⚠️ Not enough data for {param.upper()}: {len(df_clean)} rows")
            continue
        
        # Prepare features and target
        X = df_clean[feature_cols]
        y = df_clean[target_col]
        
        # Time-based split (80% train, 20% test)
        split_idx = int(0.8 * len(df_clean))
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        
        # Train XGBoost model
        model = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42,
            verbosity=0
        )
        
        model.fit(X_train, y_train)
        
        # Evaluate
        y_pred_test = model.predict(X_test)
        test_r2 = r2_score(y_test, y_pred_test)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        
        # Store results
        simple_models[param] = {
            'model': model,
            'target_col': target_col,
            'feature_cols': feature_cols,
            'test_r2': test_r2,
            'test_mae': test_mae,
            'is_log_transformed': '_log' in target_col,
            'train_samples': len(X_train),
            'test_samples': len(X_test)
        }
        
        print(f"   Features: {len(feature_cols)}")
        print(f"   Train samples: {len(X_train)}")
        print(f"   Test R²: {test_r2:.4f}")
        print(f"   Test MAE: {test_mae:.4f}")
        print(f"   ✅ Completed")
    
    return simple_models

# Train the models
simple_models = train_simple_lag_models(lag_dfs, params)

In [ ]:
# Performance Summary and Prediction Function
print("\n📊 SIMPLE LAG MODEL PERFORMANCE SUMMARY:")
print("-" * 70)
print(f"{'Parameter':<8} {'R²':<8} {'MAE':<8} {'Features':<8} {'Log Trans':<10}")
print("-" * 70)

for param, results in simple_models.items():
    log_status = "Yes" if results['is_log_transformed'] else "No"
    print(f"{param.upper():<8} {results['test_r2']:<8.4f} {results['test_mae']:<8.4f} "
          f"{len(results['feature_cols']):<8} {log_status:<10}")

def predict_next_hour(param, current_weather, lag_weather=None, lag_param=None, 
                     current_time=None, models=simple_models, scalers=scalers):
    """
    Predict next hour's pollutant concentration
    
    Args:
        param: Parameter to predict ('co', 'no2', etc.)
        current_weather: Dict of current weather {feature: value}
        lag_weather: Dict of 1-hour ago weather (optional, uses current if None)
        lag_param: Previous hour's parameter value (optional, uses 0 if None)
        current_time: Current datetime (optional, uses now if None)
    """
    if param not in models:
        return f"No model available for {param}"
    
    if current_time is None:
        current_time = pd.Timestamp.now()
    
    if lag_weather is None:
        lag_weather = current_weather.copy()
    
    if lag_param is None:
        lag_param = 0.0
    
    model_info = models[param]
    model = model_info['model']
    scaler_info = scalers[param]
    
    # Scale current weather features
    weather_features = scaler_info['feature_names']
    current_values = [current_weather.get(f, 0.0) for f in weather_features]
    lag_values = [lag_weather.get(f, 0.0) for f in weather_features]
    
    if scaler_info['scaler']:
        current_scaled = scaler_info['scaler'].transform([current_values])[0]
        lag_scaled = scaler_info['scaler'].transform([lag_values])[0]
    else:
        current_scaled = current_values
        lag_scaled = lag_values
    
    # Prepare feature vector
    feature_vector = []
    
    # Current weather (scaled)
    for i, feature in enumerate(weather_features):
        feature_vector.append(current_scaled[i])
    
    # Lag weather (scaled)
    for i, feature in enumerate(weather_features):
        feature_vector.append(lag_scaled[i])
    
    # Lag parameter value (log-transformed if needed)
    if model_info['is_log_transformed'] and lag_param > 0:
        lag_param_processed = np.log(lag_param)
    else:
        lag_param_processed = lag_param
    feature_vector.append(lag_param_processed)
    
    # Time features
    feature_vector.extend([
        current_time.hour,
        current_time.dayofweek,
        current_time.month,
        1 if current_time.dayofweek >= 5 else 0  # is_weekend
    ])
    
    # Make prediction
    prediction = model.predict([feature_vector])[0]
    
    # Back-transform if log-transformed
    if model_info['is_log_transformed']:
        prediction = np.exp(prediction)
    
    return max(0, prediction)  # Ensure non-negative

# Example prediction
example_weather = {
    'temp': 25.0, 'dwpt': 18.0, 'rhum': 65.0, 'prcp': 0.0, 'snow': 0.0,
    'wdir': 180.0, 'wspd': 10.0, 'wpgt': 15.0, 'pres': 1013.25, 'tsun': 8.0, 'coco': 3.0
}

print(f"\n🔮 EXAMPLE PREDICTIONS (next hour):")
print("Weather conditions: Temp=25°C, Wind=10km/h, Humidity=65%")
print("-" * 50)

for param in ['co', 'no2', 'pm25', 'o3']:  # Example subset
    if param in simple_models:
        pred = predict_next_hour(param, example_weather, lag_param=20.0)
        print(f"{param.upper():<6}: {pred:.4f} μg/m³")

print(f"\n✅ Simple lag models ready for prediction!")
print(f"📝 Use predict_next_hour(param, weather_dict) to make predictions")